In [1]:
# --- PASO 1: Importar bibliotecas ---
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score

# --- PASO 2: Cargar datos y dividir en train/validation/test ---
# Cargamos el dataset Iris (150 muestras, 4 características, 3 clases)
data = load_iris()
X, y = data.data, data.target

# División estratificada (60% train, 20% validation, 20% test)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, 
    test_size=0.25,  # 0.25 * 80% = 20% del total
    stratify=y_temp,
    random_state=42
)

print("Tamaños de conjuntos:")
print(f"- Train: {X_train.shape[0]} muestras ({len(y_train)/len(y)*100:.0f}%)")
print(f"- Validation: {X_val.shape[0]} muestras ({len(y_val)/len(y)*100:.0f}%)")
print(f"- Test: {X_test.shape[0]} muestras ({len(y_test)/len(y)*100:.0f}%)")

# --- PASO 3: Entrenamiento sin pipeline (con posible data leakage) ---
# ¡ERROR COMÚN: Preprocesar antes de dividir!
scaler = StandardScaler().fit(X_train)  # Aprende parámetros SOLO con train
X_train_scaled = scaler.transform(X_train)
X_val_scaled = scaler.transform(X_val)  # Aplica misma transformación

model = LogisticRegression(max_iter=200)
model.fit(X_train_scaled, y_train)
y_pred_val = model.predict(X_val_scaled)

print("\nPrecisión en validation (sin pipeline):", 
      accuracy_score(y_val, y_pred_val))

# --- PASO 4: Entrenamiento con pipeline (correcto) ---
# Pipeline integra preprocesamiento y modelo
pipe = make_pipeline(
    StandardScaler(),         # Paso 1: Estandarización
    LogisticRegression(max_iter=200)  # Paso 2: Modelo
)

pipe.fit(X_train, y_train)  # .fit_transform() en escaler + .fit() en modelo

# Predicciones y evaluación
y_pred_val_pipe = pipe.predict(X_val)
y_pred_test_pipe = pipe.predict(X_test)

print("Precisión en validation (con pipeline):", 
      accuracy_score(y_val, y_pred_val_pipe))
print("Precisión en test (métrica final):", 
      accuracy_score(y_test, y_pred_test_pipe))

# --- PASO 5: Análisis de resultados ---
print("\n--- Lecciones aprendidas ---")
print("1. La precisión en test es la única métrica confiable para producción")
print("2. El pipeline evita data leakage al encapsular el preprocesamiento")
print(f"3. El modelo generaliza bien? Test vs Validation: {accuracy_score(y_test, y_pred_test_pipe):.2f} vs {accuracy_score(y_val, y_pred_val_pipe):.2f}")

Tamaños de conjuntos:
- Train: 90 muestras (60%)
- Validation: 30 muestras (20%)
- Test: 30 muestras (20%)

Precisión en validation (sin pipeline): 0.9333333333333333
Precisión en validation (con pipeline): 0.9333333333333333
Precisión en test (métrica final): 0.9333333333333333

--- Lecciones aprendidas ---
1. La precisión en test es la única métrica confiable para producción
2. El pipeline evita data leakage al encapsular el preprocesamiento
3. El modelo generaliza bien? Test vs Validation: 0.93 vs 0.93
